In [1]:
import matstat

import numpy as np

In [2]:
data = np.array([
    (None, 20, 10), 
    (20, 24, 20),
    (24, 28, 50),
    (28, 32, 15),
    (32, None, 5)
                ])

In [3]:
ivs = matstat.intervalVariationSeries(data)

In [4]:
ivs.table

,left,right,x,n,h,w,n/h,w/h,w_cum
0,16.0,20.0,18.0,10.0,4.0,0.10,2.5,0.025,0.10
1,20.0,24.0,22.0,20.0,4.0,0.20,5.0,0.05,0.30
2,24.0,28.0,26.0,50.0,4.0,0.50,12.5,0.125,0.80
3,28.0,32.0,30.0,15.0,4.0,0.15,3.75,0.0375,0.95
4,32.0,36.0,34.0,5.0,4.0,0.05,1.25,0.0125,1.00
Total,120.0,140.0,130.0,100.0,20.0,1.00,25.0,0.25,3.15


In [5]:
ivs.show_stats()

,mean,mode,median
0,25.4,25.8462,25.6


In [6]:
disc1_dist.table

NameError: name 'disc1_dist' is not defined

### Frequency poligon

In [ ]:
disc1_dist.plot_polygon()

### Empirical distribution function (Fx)

In [ ]:
disc1_dist.plot_Fx()

In [ ]:
# TODO: idiotic distribution. make it one list with tuples for this task => [(i, n_i), ...]

In [ ]:
disc2 = np.array([rng.choice(np.arange(0, 10), replace=False, size=(5)), rng.integers(0, 50, 5)])
disc2[0].sort()

In [ ]:
disc2_dist = DVS(disc2)

disc2_dist.table

### Frequency polygon

In [ ]:
disc2_dist.plot_polygon()

### Empirical distribution function (Fx)

In [ ]:
disc2_dist.plot_Fx()